# Notebook 1: Homology matrix generation from genome sequences

Generate strain-specific model for PmegMOA

1. Perform BLASTp to find homologous proteins in strains of interest
2. Use best bidirectional hits to create gene presence/absence matrix
3. Supplementary for best practice: use BLASTn to check if we have missed any unannotated open reading frames and retain these genes in orthology matrix as well as guide future manual curation

In [1]:
#import packages needed
import pandas as pd
from glob import glob
from Bio import Entrez, SeqIO

### Generate FASTA files for both Protein and Nucleotide Pipelines
From the GenBank file, we can extract sequence and annoation information to generate fasta files for the protein and nucleotide analyses. The resulting fasta files will then be used in step 2 as input for BLAST 

In [2]:
referenceStrainID = 'Bsubt2'
targetStrainID = 'PmegMOA'

In [3]:
# define a function to parse the Genbank file to generate fasta files for both protein and nucleotide sequences
def parse_genome(id, type = 'prot', in_folder = 'genomes', out_folder = 'prots', overwrite = 1):

    in_file = '%s/%s.gb'%(in_folder, id)
    out_file = '%s/%s.fa'%(out_folder, id)
    files = glob('%s/*.fa'%out_folder)
    
    if out_file in files and overwrite == 0:
        print (out_file, 'already parsed')
        return
    else:
        print ('parsing %s'%id)
    
    handle = open(in_file)
    
    fout = open(out_file, 'w')
    x = 0
    
    records = SeqIO.parse(handle, "genbank")
    for record in records:
        for f in record.features:
            if f.type == 'CDS':
                seq = f.extract(record.seq)
                
                if type == 'nucl':
                    seq = str(seq)
                else:
                    seq = str(seq.translate())
                    
                if 'locus_tag' in f.qualifiers.keys():
                    locus = f.qualifiers['locus_tag'][0]
                elif 'gene' in f.qualifiers.keys():
                    locus = f.qualifiers['gene'][0]
                else:
                    locus = 'gene_%i'%x
                    x += 1
                fout.write('>%s\n%s\n'%(locus, seq))
    fout.close()

In [4]:
# Generate fasta files for PmegMOA

parse_genome(targetStrainID, type = 'prot', in_folder = 'genomes', out_folder = 'prots')
parse_genome(targetStrainID, type = 'nucl', in_folder = 'genomes', out_folder = 'nucl')

parsing PmegMOA
parsing PmegMOA


In [5]:
#Also generate fasta files for the reference strain

parse_genome(referenceStrainID, type = 'nucl', in_folder = 'genomes', out_folder = 'nucl')
parse_genome(referenceStrainID, type = 'prots', in_folder = 'genomes', out_folder = 'prots')

parsing Bsubt2
parsing Bsubt2


## 1. Perform BLAST to find homologous proteins in strains of interest

Make BLAST DB for each of the target strains for both Protein and Nucleotide Pipelines

In this tutorial, we will run both BLASTp for proteins and BLASTn for nucleotides. BLASTp will be used as the main approach to identify homologous proteins in reference strain and other strains of interest, while BLASTn will be used as a supplementary method to check for any unannotated genes


In [6]:
# Define a function to make blast database for either protein of nucleotide
def make_blast_db(id, folder = 'prots', db_type = 'prot'):
    import os
    
    out_file = '%s/%s.fa.pin'%(folder, id)
    files = glob('%s/*.fa.pin'%folder)
    
    if out_file in files:
        print (id, 'already has a blast db')
        return
    if db_type == 'nucl':
        ext = 'fna'
    else:
        ext = 'fa'

    cmd_line = 'makeblastdb -in %s/%s.%s -dbtype %s' %(folder, id, ext, db_type)
    
    print ('making blast db with following command line...')
    print (cmd_line)
    os.system(cmd_line)

In [7]:
make_blast_db(targetStrainID, folder = 'prots', db_type = 'prot')
make_blast_db(referenceStrainID, folder = 'prots', db_type = 'prot')

PmegMOA already has a blast db
Bsubt2 already has a blast db


### Define functions to run protein BLAST and get sequence lengths
- BLASTp will be the main approach used here to identify homologous proteins between strains 
- Aside from sequence similarity, we also want to ensure the coverage of sequence mapping is sufficient. Therefore, we need to identiy the sequence length for each protein and compare it with the alignment length.

In [8]:
# define a function to run BLASTp
def run_blastp(seq, db, in_folder = 'prots', out_folder = 'bbh', out = None, outfmt = 6, evalue = 0.001, threads = 1):
    import os
    if out == None:
        out = '%s/%s_vs_%s.txt'%(out_folder, seq, db)
        print(out)
    
    files = glob('%s/*.txt'%out_folder)
    if out in files:
        print (seq, 'already blasted')
        return
    
    print('blasting %s vs %s'%(seq, db))
    
    db = '%s/%s.fa'%(in_folder, db)
    seq = '%s/%s.fa'%(in_folder, seq)
    cmd_line = 'blastp -db %s -query %s -out %s -evalue %s -outfmt %s -num_threads %i' \
    %(db, seq, out, evalue, outfmt, threads)
    
    print ('running blastp with following command line...')
    print (cmd_line)
    os.system(cmd_line)
    return out

In [9]:
# define a function to get sequence length 

def get_gene_lens(query, in_folder = 'prots'):

    file = '%s/%s.fa'%(in_folder, query)
    handle = open(file)
    records = SeqIO.parse(handle, "fasta")
    out = []
    
    for record in records:
        out.append({'gene':record.name, 'gene_length':len(record.seq)})
    
    out = pd.DataFrame(out)
    return out

## 2. Use Bi-Directional BLASTp Best Hits to create gene presence/absence matrix

### Obtain Bi-Directional BLASTp Best Hits

From the above BLASTp results, we can obtain Bi-Directional BLASTp Best Hits to identify homologous proteins. Note beside gene similarity score, the coverage of alignment is also used to filter mapping results. 

In [10]:
# define a function to get Bi-Directional BLASTp Best Hits
def get_bbh(query, subject, in_folder = 'bbh'):    
    
    #Utilize the defined protein BLAST function
    run_blastp(query, subject)
    run_blastp(subject, query)
    
    query_lengths = get_gene_lens(query, in_folder = 'prots')
    subject_lengths = get_gene_lens(subject, in_folder = 'prots')
    
    #Define the output file of this BLAST
    out_file = '%s/%s_vs_%s_parsed.csv'%(in_folder,query, subject)
    files = glob('%s/*_parsed.csv'%in_folder)
    
    #Combine the results of the protein BLAST into a dataframe
    print ('parsing BBHs for', query, subject)
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    bbh = pd.read_csv('%s/%s_vs_%s.txt'%(in_folder, query, subject), sep = '\t', names = cols)
    bbh = pd.merge(bbh, query_lengths) 
    bbh['COV'] = bbh['alnLength']/bbh['gene_length']
    
    bbh2 = pd.read_csv('%s/%s_vs_%s.txt'%(in_folder, subject, query), sep = '\t', names = cols)
    bbh2 = pd.merge(bbh2, subject_lengths) 
    bbh2['COV'] = bbh2['alnLength']/bbh2['gene_length']
    out = pd.DataFrame()
    
    # Filter the genes based on coverage
    bbh = bbh[bbh.COV >= 0.25]
    bbh2 = bbh2[bbh2.COV >= 0.25]
    
    #Delineate the best hits from the BLAST
    for g in bbh.gene.unique():
        res = bbh[bbh.gene == g]
        if len(res) == 0:
            continue
        best_hit = res.loc[res.PID.idxmax()]
        best_gene = best_hit.subject
        res2 = bbh2[bbh2.gene == best_gene]
        if len(res2) == 0:
            continue
        best_hit2 = res2.loc[res2.PID.idxmax()]
        best_gene2 = best_hit2.subject
        if g == best_gene2:
            best_hit['BBH'] = '<=>'
        else:
            best_hit['BBH'] = '->'
        out = pd.concat([out, pd.DataFrame(best_hit).transpose()])
    
    #Save the final file to a designated CSV file    
    out.to_csv(out_file)

In [11]:
# Execute the BLAST for each target strain against the reference strain, save results to 'bbh' i.e. "bidirectional best
# hits" folder to create homology matrix

get_bbh(referenceStrainID, targetStrainID, in_folder = 'bbh')

bbh/Bsubt2_vs_PmegMOA.txt
Bsubt2 already blasted
bbh/PmegMOA_vs_Bsubt2.txt
PmegMOA already blasted
parsing BBHs for Bsubt2 PmegMOA


/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

...

/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '->'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_hit['BBH'] = '<=>'
/tmp/ipykernel_12507/1865688047.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

### Parse the BLAST Results into one Homology Matrix of the Reconstruction Genes

For the homology matrix, we only focus on genes that are present in the reference model

In [12]:
#Load all the BLAST files between the reference strain and target strains

blast_files = glob('%s/*_parsed.csv'%'bbh')

for blast in blast_files:
    bbh = pd.read_csv(blast)
    print(blast, bbh.shape)

bbh/Bsubt2_vs_PmegMOA_parsed.csv (3077, 16)


In [13]:
import cobra
model = cobra.io.read_sbml_model('iYO844_FixedIDs.xml')
listGeneIDs = []

for gene in model.genes:
    listGeneIDs.append(gene.id)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-18


In [14]:
listGeneIDs

['BSU_30820',
 'BSU_16740',
 'BSU_16730',
 'BSU_06310',
 'BSU_36710',
 'BSU_12160',
 'BSU_27220',
 'BSU_18570',
 'BSU_08860',
 'BSU_03040',
 'BSU_03900',
 'BSU_39670',
 'BSU_37120',
 'BSU_40190',
 'BSU_10130',
 'BSU_38560',
 'BSU_40110',
 'BSU_38830',
 'BSU_39860',
 'BSU_19310',
 'BSU_32830',
 'BSU_24170',
 'BSU_29200',
 'BSU_24350',
 'BSU_24340',
 'BSU_22440',
 'BSU_29210',
 'BSU_07700',
 'BSU_13900',
 'BSU_13910',
 'BSU_11210',
 'BSU_01620',
 'BSU_01610',
 'BSU_01630',
 'BSU_11200',
 'BSU_00770',
 'BSU_29470',
 'BSU_36000',
 'BSU_15350',
 'BSU_32820',
 'BSU_18260',
 'BSU_04520',
 'BSU_18000',
 'BSU_37170',
 'BSU_24150',
 'BSU_11220',
 'BSU_04620',
 'BSU_14180',
 'BSU_23280',
 'BSU_15570',
 'BSU_19230',
 'BSU_29680',
 'BSU_09410',
 'BSU_05740',
 'BSU_14000',
 'BSU_09530',
 'BSU_39780',
 'BSU_08090',
 'BSU_08070',
 'BSU_29690',
 'BSU_08080',
 'BSU_29700',
 'BSU_29710',
 'BSU_08060',
 'BSU_30420',
 'BSU_30440',
 'BSU_30450',
 'BSU_30410',
 'BSU_30430',
 'BSU_26970',
 'BSU_27010',
 'BSU_

In [15]:
#Create 2 matrices of N, rows where N is the number of model genes and M columns where M is the number of target strains
#One matrix will be populated with the PID results from the blasts and another with the mapping of gene locus tags

ortho_matrix = pd.DataFrame(index = listGeneIDs, columns = [targetStrainID])
geneIDs_matrix = pd.DataFrame(index = listGeneIDs, columns = [targetStrainID])

In [16]:
#Parse through each blast file and acquire pertinent information for each matrix for each of the base reconstruction genes
for blast in blast_files:
    bbh = pd.read_csv(blast)
    listIDs = []
    listPID = []
    for r, row in ortho_matrix.iterrows():
        print(r, row)
        try:
            currentOrtholog = bbh[bbh['gene'] == r].reset_index()
            listIDs.append(currentOrtholog.iloc[0]['subject'])
            listPID.append(currentOrtholog.iloc[0]['PID'])
        except:
            listIDs.append('None')
            listPID.append(0)
    for col in ortho_matrix.columns:
        if col in blast:
            ortho_matrix[col] = listPID
            geneIDs_matrix[col] = listIDs

BSU_30820 PmegMOA    NaN
Name: BSU_30820, dtype: object
BSU_16740 PmegMOA    NaN
Name: BSU_16740, dtype: object
BSU_16730 PmegMOA    NaN
Name: BSU_16730, dtype: object
BSU_06310 PmegMOA    NaN
Name: BSU_06310, dtype: object
BSU_36710 PmegMOA    NaN
Name: BSU_36710, dtype: object
BSU_12160 PmegMOA    NaN
Name: BSU_12160, dtype: object
BSU_27220 PmegMOA    NaN
Name: BSU_27220, dtype: object
BSU_18570 PmegMOA    NaN
Name: BSU_18570, dtype: object
BSU_08860 PmegMOA    NaN
Name: BSU_08860, dtype: object
BSU_03040 PmegMOA    NaN
Name: BSU_03040, dtype: object
BSU_03900 PmegMOA    NaN
Name: BSU_03900, dtype: object
BSU_39670 PmegMOA    NaN
Name: BSU_39670, dtype: object
BSU_37120 PmegMOA    NaN
Name: BSU_37120, dtype: object
BSU_40190 PmegMOA    NaN
Name: BSU_40190, dtype: object
BSU_10130 PmegMOA    NaN
Name: BSU_10130, dtype: object
BSU_38560 PmegMOA    NaN
Name: BSU_38560, dtype: object
BSU_40110 PmegMOA    NaN
Name: BSU_40110, dtype: object
BSU_38830 PmegMOA    NaN
Name: BSU_38830, dtype:

BSU_33180 PmegMOA    NaN
Name: BSU_33180, dtype: object
BSU_33160 PmegMOA    NaN
Name: BSU_33160, dtype: object
BSU_33170 PmegMOA    NaN
Name: BSU_33170, dtype: object
BSU_26650 PmegMOA    NaN
Name: BSU_26650, dtype: object
BSU_11240 PmegMOA    NaN
Name: BSU_11240, dtype: object
BSU_15520 PmegMOA    NaN
Name: BSU_15520, dtype: object
BSU_11230 PmegMOA    NaN
Name: BSU_11230, dtype: object
BSU_15510 PmegMOA    NaN
Name: BSU_15510, dtype: object
BSU_27330 PmegMOA    NaN
Name: BSU_27330, dtype: object
BSU_13850 PmegMOA    NaN
Name: BSU_13850, dtype: object
BSU_33490 PmegMOA    NaN
Name: BSU_33490, dtype: object
BSU_16920 PmegMOA    NaN
Name: BSU_16920, dtype: object
BSU_02270 PmegMOA    NaN
Name: BSU_02270, dtype: object
BSU_00460 PmegMOA    NaN
Name: BSU_00460, dtype: object
BSU_30240 PmegMOA    NaN
Name: BSU_30240, dtype: object
BSU_32180 PmegMOA    NaN
Name: BSU_32180, dtype: object
BSU_39410 PmegMOA    NaN
Name: BSU_39410, dtype: object
BSU_39020 PmegMOA    NaN
Name: BSU_39020, dtype:

BSU_02400 PmegMOA    NaN
Name: BSU_02400, dtype: object
BSU_39390 PmegMOA    NaN
Name: BSU_39390, dtype: object
BSU_28150 PmegMOA    NaN
Name: BSU_28150, dtype: object
BSU_17150 PmegMOA    NaN
Name: BSU_17150, dtype: object
BSU_18220 PmegMOA    NaN
Name: BSU_18220, dtype: object
BSU_28540 PmegMOA    NaN
Name: BSU_28540, dtype: object
BSU_31600 PmegMOA    NaN
Name: BSU_31600, dtype: object
BSU_31660 PmegMOA    NaN
Name: BSU_31660, dtype: object
BSU_31630 PmegMOA    NaN
Name: BSU_31630, dtype: object
BSU_31650 PmegMOA    NaN
Name: BSU_31650, dtype: object
BSU_31610 PmegMOA    NaN
Name: BSU_31610, dtype: object
BSU_31620 PmegMOA    NaN
Name: BSU_31620, dtype: object
BSU_18230 PmegMOA    NaN
Name: BSU_18230, dtype: object
BSU_13960 PmegMOA    NaN
Name: BSU_13960, dtype: object
BSU_11710 PmegMOA    NaN
Name: BSU_11710, dtype: object
BSU_38020 PmegMOA    NaN
Name: BSU_38020, dtype: object
BSU_37780 PmegMOA    NaN
Name: BSU_37780, dtype: object
BSU_31322 PmegMOA    NaN
Name: BSU_31322, dtype:

BSU_23770 PmegMOA    NaN
Name: BSU_23770, dtype: object
BSU_15630 PmegMOA    NaN
Name: BSU_15630, dtype: object
BSU_06500 PmegMOA    NaN
Name: BSU_06500, dtype: object
BSU_34860 PmegMOA    NaN
Name: BSU_34860, dtype: object
BSU_06450 PmegMOA    NaN
Name: BSU_06450, dtype: object
BSU_06480 PmegMOA    NaN
Name: BSU_06480, dtype: object
BSU_06470 PmegMOA    NaN
Name: BSU_06470, dtype: object
BSU_06460 PmegMOA    NaN
Name: BSU_06460, dtype: object
BSU_34880 PmegMOA    NaN
Name: BSU_34880, dtype: object
BSU_15850 PmegMOA    NaN
Name: BSU_15850, dtype: object
BSU_15860 PmegMOA    NaN
Name: BSU_15860, dtype: object
BSU_15620 PmegMOA    NaN
Name: BSU_15620, dtype: object
BSU_15550 PmegMOA    NaN
Name: BSU_15550, dtype: object
BSU_25660 PmegMOA    NaN
Name: BSU_25660, dtype: object
BSU_03150 PmegMOA    NaN
Name: BSU_03150, dtype: object
BSU_11870 PmegMOA    NaN
Name: BSU_11870, dtype: object
BSU_32440 PmegMOA    NaN
Name: BSU_32440, dtype: object
BSU_32430 PmegMOA    NaN
Name: BSU_32430, dtype:

### Apply Similarity Threshold to Binarize  Homology Matrix to Presence/Absence Matrix
In this step, choose a threshold for the PID to determine if a gene is a absent/present in the strain of interest. We can then convert the homology matrix generated above into a binarized presence/absence matrix

In [17]:
# Genes with a greater than 60% PID are considered present in the target strain genome 
# and consequently less than 60% are considered absent from the target strain genome
for column in ortho_matrix:
    ortho_matrix.loc[ortho_matrix[column] <= 60.0,column] = 0
    ortho_matrix.loc[ortho_matrix[column] > 60.0,column] = 1

In [18]:
ortho_matrix

,PmegMOA
BSU_30820,0.0
BSU_16740,1.0
BSU_16730,1.0
BSU_06310,0.0
BSU_36710,0.0
...,...
BSU_23860,1.0
BSU_33930,1.0
BSU_09240,0.0
BSU_33910,1.0


## 3. Perform BLASTn to check unannotated open reading frames to guide manual curation 
At this juncture it may be useful to execute a supplementary nucleotide BLAST to check for unannotated genes, results here become candidates for manual curation. In this tutorial we retain unannotated genes that pass the threhsold for
similarity and contain no premature stop codons

In [19]:
#Define a function to generate FNA from the GBK files

def gbk2fasta(gbk_filename):
    faa_filename = '.'.join(gbk_filename.split('.')[:-1])+'.fna'
    input_handle  = open(gbk_filename, "r")
    output_handle = open(faa_filename, "w")

    for seq_record in SeqIO.parse(input_handle, "genbank") :
        print ("Converting GenBank record %s" % seq_record.id)
        output_handle.write(">%s %s\n%s\n" % (
               seq_record.id,
               seq_record.description,
               seq_record.seq))

    output_handle.close()
    input_handle.close()

In [20]:
#Define function to run the BLASTn

def run_blastn(seq, db, outfmt = 6, evalue = 0.001,threads = 1):
    import os
    out = 'nucl/'+seq+'_vs_'+db+'.txt'
    seq = 'nucl/'+seq+'.fa'
    db = 'genomes/'+db+'.fna'
    
    cmd_line = 'blastn -db %s -query %s -out %s -evalue %s -outfmt %s -num_threads %i' \
    %(db, seq, out, evalue, outfmt, threads)
    
    print ('running blastn with following command line...')
    print (cmd_line)
    os.system(cmd_line)
    return out

In [21]:
# convert genbank files to fna files for strains of interest

gbk2fasta('genomes/'+targetStrainID+'.gb')

Converting GenBank record NODE_1_length_132867_cov_126.236720
Converting GenBank record NODE_2_length_103972_cov_145.337677
Converting GenBank record NODE_3_length_98070_cov_142.751472
Converting GenBank record NODE_4_length_97076_cov_120.918257
Converting GenBank record NODE_5_length_93380_cov_122.726547
Converting GenBank record NODE_6_length_84183_cov_123.491630
Converting GenBank record NODE_7_length_77487_cov_135.428446
Converting GenBank record NODE_8_length_72864_cov_154.944867
Converting GenBank record NODE_9_length_68904_cov_166.400657
Converting GenBank record NODE_10_length_64414_cov_278.936102
Converting GenBank record NODE_11_length_62755_cov_125.58282
Converting GenBank record NODE_12_length_59855_cov_136.825220
Converting GenBank record NODE_13_length_58972_cov_159.635436
Converting GenBank record NODE_14_length_58649_cov_164.143806
Converting GenBank record NODE_15_length_57890_cov_144.201287
Converting GenBank record NODE_16_length_54124_cov_156.61039
Converting GenBan

In [22]:
# make nucleotide sequence databases

make_blast_db(targetStrainID, folder = 'genomes', db_type = 'nucl')

making blast db with following command line...
makeblastdb -in genomes/PmegMOA.fna -dbtype nucl


Building a new DB, current time: 05/22/2023 15:30:54
New DB name:   /home/hugo/Desktop/TFM_Bsubtilis/3_Draft Construction/genomes/PmegMOA.fna
New DB title:  genomes/PmegMOA.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/hugo/Desktop/TFM_Bsubtilis/3_Draft Construction/genomes/PmegMOA.fna
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 520 sequences in 0.0391829 seconds.




In [23]:
# perform uni-directional BLASTn hit

genome_blast_res = []
res = run_blastn(referenceStrainID,targetStrainID)
genome_blast_res.append(res)

running blastn with following command line...
blastn -db genomes/PmegMOA.fna -query nucl/Bsubt2.fa -out nucl/Bsubt2_vs_PmegMOA.txt -evalue 0.001 -outfmt 6 -num_threads 1


In [24]:
#define a function to parse through the nucleotide BLAST results and form one matrix of all the results

def parse_nucl_blast(infile):
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    data = pd.read_csv(infile, sep = '\t', names = cols)
    data = data[(data['PID'] > 60) & (data['alnLength'] > 0.8 * data['queryEnd'])]
    data2 = data.groupby('gene').first()
    return data2.reset_index()

In [25]:
'''
# parse the nucleotide blast matrix

na_matrix=pd.DataFrame()

for file in genome_blast_res:
    genes =parse_nucl_blast(file)
    name ='.'.join(file.split('_')[-1].split('.')[:-1])
    na_matrix = na_matrix.append(genes[['gene','subject','PID']])
na_matrix = pd.pivot_table(na_matrix, index='gene', columns='subject',values='PID')

na_matrix.head()
'''

"\n# parse the nucleotide blast matrix\n\nna_matrix=pd.DataFrame()\n\nfor file in genome_blast_res:\n    genes =parse_nucl_blast(file)\n    name ='.'.join(file.split('_')[-1].split('.')[:-1])\n    na_matrix = na_matrix.append(genes[['gene','subject','PID']])\nna_matrix = pd.pivot_table(na_matrix, index='gene', columns='subject',values='PID')\n\nna_matrix.head()\n"

In [26]:
# parse the nucleotide blast matrixna_matrix = pd.DataFrame()

na_matrix=pd.DataFrame()
file = 'nucl/Bsubt2_vs_PmegMOA.txt'

genes = parse_nucl_blast(file)
name = '.'.join(file.split('_')[-1].split('.')[:-1])
na_matrix = na_matrix.append(genes[['gene','subject','PID']])

na_matrix = pd.pivot_table(na_matrix, index = 'gene', values = 'PID')

/tmp/ipykernel_12507/63319243.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  na_matrix = na_matrix.append(genes[['gene','subject','PID']])


In [27]:
na_matrix.head()

,PID
gene,
BSU_00020,74.635
BSU_00090,77.351
BSU_00320,73.009
BSU_00370,83.568
BSU_00750,73.601


### Examine unnannotated open reading frames
We compare the results from BLASTp and BLASTn and record any inconsistencies between the two matrices due to missing annotation. This result is then saved to guide future manual curation. 

In [28]:
# define a function to extract the sequence from fna file 

def extract_seq(g, contig, start, end):
    from Bio import SeqIO
    handle = open(g)
    records = SeqIO.parse(handle, "fasta")
    
    for record in records:
        if record.name == contig:
            if end > start:
                section = record[start:end]
            else:
                section = record[end-1:start+1].reverse_complement()
                
            seq = str(section.seq)
    return seq

In [29]:
#Define updated matrices that will include genes based on sequence evidence that were missing due to lack of annotation

ortho_matrix_w_unannotated = ortho_matrix.copy()
geneIDs_matrix_w_unannotated = geneIDs_matrix.copy()

In [30]:
#Define matrix of the BLASTn results for all the pertinent model genes

nonModelGenes=[]
for g in na_matrix.index:
    if g not in listGeneIDs:
        nonModelGenes.append(g)

na_model_genes=na_matrix.drop(nonModelGenes)

In [31]:
na_model_genes

,PID
gene,
BSU_00090,77.351
BSU_00750,73.601
BSU_01370,74.937
BSU_11640,71.510
BSU_14590,80.816
BSU_17460,79.598
BSU_22710,72.399
BSU_30720,73.179
BSU_33440,73.478


In [32]:
#For each strain in the ortho_matrix, identify genes that meet threshold of SEQ similarity, but missing from
#annotated ORFS. Additionally, look at the sequence to ensure that these cases do not have early stop codons indicating
#nonfunctional even if the NA seqs meet the threshold

pseudogenes = {}

for c in ortho_matrix.columns:
    
    orfs = ortho_matrix[c]
    genes = na_model_genes['PID']
    # All the Model Genes that met the BLASTp Requirements
    orfs2 = orfs[orfs == 1].index.tolist()
    # All the Model Genes based off of BLASTn similarity above threshold of 80
    genes2 = genes[genes>=80].index.tolist()
    # By Definition find the genes that pass sequence threshold but were NOT in annotated ORFs:
    unannotated = set(genes2) -set(orfs2)
    
    # Obtain sequences of this list to check for premature stop codons:
    data = 'nucl/Bsubt2_vs_%s.txt'%c
    cols = ['gene', 'subject', 'PID', 'alnLength', 'mismatchCount', 'gapOpenCount', 'queryStart', 'queryEnd', 'subjectStart', 'subjectEnd', 'eVal', 'bitScore']
    data = pd.read_csv(data, sep='\t', names=cols)
    #
    pseudogenes[c] = {}
    unannotated_data = data[data['gene'].isin(list(unannotated))]
    for i in unannotated_data.index:
        gene = data.loc[i,'gene']
        contig = data.loc[i,'subject'] 
        start = data.loc[i,'subjectStart']
        end = data.loc[i,'subjectEnd']
        seq = extract_seq('genomes/%s.fna'%c,contig, start, end)
        # check for early stop codons - these are likely nonfunctional and shouldn't be included
        if '*' in seq:
            print (seq)
            pseudogenes[c][gene]=seq
            # Remove the gene from list of unannotated genes
            unannotated-set([gene])
            
    print (c, unannotated)
    
    # For pertinent genes, retain those based off of nucleotide similarity within the orthology matrix and geneIDs matrix
    ortho_matrix_w_unannotated.loc[unannotated,c]=1
    for g in unannotated:
        geneIDs_matrix_w_unannotated.loc[g,c] = '%s_ortholog'%g
    

PmegMOA set()


/tmp/ipykernel_12507/823313240.py:41: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  ortho_matrix_w_unannotated.loc[unannotated,c]=1


In [33]:
#Save the Presence/Absence Matrix and geneIDs Matrix for future use

ortho_matrix_w_unannotated.to_csv('ortho_matrix.csv')
geneIDs_matrix_w_unannotated.to_csv('geneIDs_matrix.csv')

# Notebook 2: Generate multi-strain models

This notebook follows Notebook 1 in the tutorial, and continues to work on generating multi-strain E.coli models. This notebook utilizes the output of notebook 1 (presence/absence matrix and geneID matrix) to generate draft strain-specific models from the 
reference model. There are two major steps involved: 

1. Deletion of missing genes/reaction from reference model to generate draft models
2. Update gene-protein-reaction rule in each model

In [34]:
#import package needed
import cobra
import pandas as pd
from cobra.io import read_sbml_model
from glob import glob
from cobra.manipulation.delete import delete_model_genes, remove_genes

## 1. Deletion of missing genes/reaction from reference model 

In [35]:
#Load the base E. coli reconstruction iML1515

model = read_sbml_model('iYO844_FixedIDs.xml')
model

Name,default_model
Memory address,7fd460569d00
Number of metabolites,990
Number of reactions,1250
Number of genes,844
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"c, e"


In [36]:
## Load the previously generated homology matrix for E. coli strains of interest
hom_matrix=pd.read_csv('ortho_matrix.csv')
hom_matrix=hom_matrix.set_index('Unnamed: 0')

In [37]:
hom_matrix

,PmegMOA
Unnamed: 0,
BSU_30820,0.0
BSU_16740,1.0
BSU_16730,1.0
BSU_06310,0.0
BSU_36710,0.0
...,...
BSU_23860,1.0
BSU_33930,1.0
BSU_09240,0.0


### Delete missing genes from copies of the _B. subtilis_ str. 168 model
For each strain, start with the _B. subtilis_ str. 168 model, identify the missing genes from the matrix, and remove them and their associated reactions from the reference model 

In [38]:
#create strain-specific draft models and save them
for strain in hom_matrix.columns:
    
    #Get the list of Gene IDs from the homology matrix dataframe for the current strain without a homolog
    currentStrain = hom_matrix[strain]
    nonHomologous = currentStrain[currentStrain == 0.0]
    nonHomologous = nonHomologous.index.tolist()
    '''
    #s0001 is an artificial gene used in iML1515 for spontaneous reactions and as such has no homologs,
    #However, it is retained for these spontaneous reactions to function
    nonHomologous.remove('s0001')
    '''
    #Define a list of Gene objects from the base reconstruction to be deleted from the current strain
    toDelete = []
    for gene in nonHomologous:
        toDelete.append(model.genes.get_by_id(gene))

    #Establish a model copy and use the COBRApy function to remove the appropriate content and save this model
    modelCopy = model.copy()
    remove_genes(modelCopy, toDelete, remove_reactions = True)
    modelCopy.id = str(strain)
    cobra.io.json.save_json_model(modelCopy, str('Models/'+strain+'.json'), pretty = False)

Read LP format model from file /tmp/tmpnvmyow6a.lp
Reading time = 0.00 seconds
: 990 rows, 2500 columns, 10478 nonzeros


## 2. Update Model Gene Product Rules

In [39]:
#load the geneID matrix from the notebook1 

models=glob('%s/*.json'%'Models')
geneIDs_matrix=pd.read_csv('geneIDs_matrix.csv')
geneIDs_matrix=geneIDs_matrix.set_index('Unnamed: 0')
geneIDs_matrix

,PmegMOA
Unnamed: 0,
BSU_30820,NODE_131_length_8781_cov_148.559398_5
BSU_16740,NODE_3_length_98070_cov_142.751472_5
BSU_16730,NODE_3_length_98070_cov_142.751472_6
BSU_06310,NODE_106_length_12173_cov_143.203704_5
BSU_36710,None
...,...
BSU_23860,NODE_9_length_68904_cov_166.400657_66
BSU_33930,NODE_17_length_52560_cov_173.736505_14
BSU_09240,NODE_122_length_9644_cov_129.069719_6


In [40]:
#Utilize the geneIDs matrix to update the GPRs in each of the strain-specific models with the proper gene ID
# OG
from cobra.manipulation.modify import rename_genes

for mod in models:
    model=cobra.io.load_json_model(mod)
    for column in geneIDs_matrix.columns:
        if column in mod:
            currentStrain=column
    
    IDMapping=geneIDs_matrix[currentStrain].to_dict()
    IDMappingParsed = {k:v for k,v in IDMapping.items() if v != 'None'}
    
    rename_genes(model,IDMappingParsed)
    cobra.io.json.save_json_model(model,mod, pretty=False)

In [41]:
#Utilize the geneIDs matrix to update the GPRs in each of the strain-specific models with the proper gene ID

from cobra.manipulation.modify import rename_genes

for mod in models:
    model=cobra.io.load_json_model(mod)
    for column in geneIDs_matrix.columns:
        if column in mod:
            currentStrain=column
    
    IDMapping=geneIDs_matrix[currentStrain].to_dict()
    IDMappingParsed = {v:k for k,v in IDMapping.items() if v != 'None'}
    
    rename_genes(model,IDMappingParsed)
    cobra.io.json.save_json_model(model,mod, pretty=False)

In [42]:
# gather the general information on the draft models
for strain in hom_matrix.columns:
    model=cobra.io.load_json_model(str('Models/'+strain+'.json'))
    print (model.id,'Number of Model Genes:',len(model.genes),'Number of Model Reactions:',len(model.reactions))

PmegMOA Number of Model Genes: 396 Number of Model Reactions: 824


In [44]:
from cobra.io import load_json_model, write_sbml_model
final_mod = load_json_model('Models/PmegMOA.json')
write_sbml_model(final_mod, 'Models/PmegMOA_fromBsubt.xml')

In [45]:
# Save it as an .xlsx file

import importExcelModel
importExcelModel.cobrapy_to_excel(final_mod, 'Models/PmegMOA_fromBsubt.xlsx')